In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer as si
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor, AdaBoostRegressor
from sklearn.model_selection import RandomizedSearchCV, KFold, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from scipy.stats import norm

**Import datasets for training the model and testing the model with new data**

In [ ]:
players_21 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/archive/players_21.csv') # create and train model
players_22 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/archive/players_22.csv') # test model

<ipython-input-3-debe57d67d03>:2: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  players_22 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/archive/players_22.csv') # test model


In [ ]:
players_21.head()

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,103500000.0,560000.0,33,...,52+3,52+3,52+3,62+3,19+3,https://cdn.sofifa.net/players/158/023/21_120.png,https://cdn.sofifa.net/teams/241/60.png,https://cdn.sofifa.net/flags/es.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
1,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",92,92,63000000.0,220000.0,35,...,54+3,54+3,54+3,61+3,20+3,https://cdn.sofifa.net/players/020/801/21_120.png,https://cdn.sofifa.net/teams/45/60.png,https://cdn.sofifa.net/flags/it.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png
2,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,91,91,111000000.0,240000.0,31,...,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/21_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,NaN,https://cdn.sofifa.net/flags/pl.png
3,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,132000000.0,270000.0,28,...,49+3,49+3,49+3,62+3,20+3,https://cdn.sofifa.net/players/190/871/21_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/br.png
4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,"CAM, CM",91,91,129000000.0,370000.0,29,...,69+3,69+3,69+3,75+3,21+3,https://cdn.sofifa.net/players/192/985/21_120.png,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png


In [ ]:
players_21.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 110 columns):
 #    Column                       Non-Null Count  Dtype  
---   ------                       --------------  -----  
 0    sofifa_id                    18944 non-null  int64  
 1    player_url                   18944 non-null  object 
 2    short_name                   18944 non-null  object 
 3    long_name                    18944 non-null  object 
 4    player_positions             18944 non-null  object 
 5    overall                      18944 non-null  int64  
 6    potential                    18944 non-null  int64  
 7    value_eur                    18707 non-null  float64
 8    wage_eur                     18719 non-null  float64
 9    age                          18944 non-null  int64  
 10   dob                          18944 non-null  object 
 11   height_cm                    18944 non-null  int64  
 12   weight_kg                    18944 non-null  int64  
 13  

## **Preprocessing the data**

Remove the clearly useless variables according to the description of the attributes in the dataset. These attributes are not particularly usefull to the rating of a player because they are either links or they describe how players perform in other positions

In [ ]:
# take out url and id values because they are categorised as useless
useless = ['player_face_url', 'club_logo_url', 'club_flag_url', 'nation_logo_url', 'nation_flag_url', 'sofifa_id', 'player_url','gk', 'rb', 'rcb', 'cb', 'lcb', 'lb', 'rwb', 'rdm', 'cdm', 'ldm', 'lwb', 'rm', 'rcm', 'cm', 'lcm', 'lm', 'ram', 'cam', 'lam', 'rw', 'rf', 'cf', 'lf', 'lw', 'rs', 'st', 'ls']
players_21.drop(columns=useless, inplace=True)
players_21

,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,...,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed
0,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,103500000.0,560000.0,33,1987-06-24,170,...,96,32,35,24,6,11,15,14,8,NaN
1,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",92,92,63000000.0,220000.0,35,1985-02-05,187,...,95,28,32,24,7,11,15,14,11,NaN
2,R. Lewandowski,Robert Lewandowski,ST,91,91,111000000.0,240000.0,31,1988-08-21,184,...,88,35,42,19,15,6,12,8,10,NaN
3,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,132000000.0,270000.0,28,1992-02-05,175,...,93,35,30,29,9,9,15,15,11,NaN
4,K. De Bruyne,Kevin De Bruyne,"CAM, CM",91,91,129000000.0,370000.0,29,1991-06-28,181,...,91,68,65,53,15,13,5,10,13,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18939,Zhang Mengxuan,张梦炫,CB,47,52,70000.0,1000.0,21,1999-04-26,177,...,40,45,56,47,12,13,8,14,6,NaN
18940,Huang Wenzhou,黄文卓,CM,47,53,70000.0,1000.0,21,1999-01-07,174,...,35,43,42,53,8,8,13,14,10,NaN
18941,Song Yue,宋岳,CM,47,47,45000.0,2000.0,28,1991-11-20,185,...,35,38,43,45,8,5,11,5,7,NaN
18942,V. Da Silva,Ivanilson Loforte Tique Da Silva,ST,47,67,130000.0,500.0,17,2003-03-30,171,...,45,18,11,13,11,13,9,9,6,NaN


Remove attributes with 30% or more missing values

In [ ]:
null_attributes = players_21.columns[players_21.count()/18944 < 0.7]
players_21.drop(columns=null_attributes, inplace=True)

In [ ]:
players_21.shape

(18944, 69)

In [ ]:
players_21

,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,...,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
0,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,103500000.0,560000.0,33,1987-06-24,170,...,75,96,32,35,24,6,11,15,14,8
1,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",92,92,63000000.0,220000.0,35,1985-02-05,187,...,84,95,28,32,24,7,11,15,14,11
2,R. Lewandowski,Robert Lewandowski,ST,91,91,111000000.0,240000.0,31,1988-08-21,184,...,88,88,35,42,19,15,6,12,8,10
3,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,132000000.0,270000.0,28,1992-02-05,175,...,92,93,35,30,29,9,9,15,15,11
4,K. De Bruyne,Kevin De Bruyne,"CAM, CM",91,91,129000000.0,370000.0,29,1991-06-28,181,...,84,91,68,65,53,15,13,5,10,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18939,Zhang Mengxuan,张梦炫,CB,47,52,70000.0,1000.0,21,1999-04-26,177,...,35,40,45,56,47,12,13,8,14,6
18940,Huang Wenzhou,黄文卓,CM,47,53,70000.0,1000.0,21,1999-01-07,174,...,35,35,43,42,53,8,8,13,14,10
18941,Song Yue,宋岳,CM,47,47,45000.0,2000.0,28,1991-11-20,185,...,36,35,38,43,45,8,5,11,5,7
18942,V. Da Silva,Ivanilson Loforte Tique Da Silva,ST,47,67,130000.0,500.0,17,2003-03-30,171,...,50,45,18,11,13,11,13,9,9,6


Extract and Show all categorical attributes based on the data types

In [ ]:
categorical_df = players_21.select_dtypes(include=['object'])
categorical_df

,short_name,long_name,player_positions,dob,club_name,league_name,club_position,club_joined,nationality_name,preferred_foot,work_rate,body_type,real_face
0,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",1987-06-24,FC Barcelona,Spain Primera Division,CAM,2004-07-01,Argentina,Left,Medium/Low,Unique,Yes
1,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",1985-02-05,Juventus,Italian Serie A,LS,2018-07-10,Portugal,Right,High/Low,Unique,Yes
2,R. Lewandowski,Robert Lewandowski,ST,1988-08-21,FC Bayern München,German 1. Bundesliga,ST,2014-07-01,Poland,Right,High/Medium,Unique,Yes
3,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",1992-02-05,Paris Saint-Germain,French Ligue 1,LW,2017-08-03,Brazil,Right,High/Medium,Unique,Yes
4,K. De Bruyne,Kevin De Bruyne,"CAM, CM",1991-06-28,Manchester City,English Premier League,RCM,2015-08-30,Belgium,Right,High/High,Unique,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18939,Zhang Mengxuan,张梦炫,CB,1999-04-26,Chongqing Liangjiang Athletic,Chinese Super League,SUB,2020-08-01,China PR,Right,Low/Low,Normal (170-185),No
18940,Huang Wenzhou,黄文卓,CM,1999-01-07,Shanghai Port FC,Chinese Super League,RES,2020-08-01,China PR,Right,Low/Low,Lean (170-185),No
18941,Song Yue,宋岳,CM,1991-11-20,Tianjin Jinmen Tiger FC,Chinese Super League,RES,2020-08-01,China PR,Right,Low/Low,Lean (185+),No
18942,V. Da Silva,Ivanilson Loforte Tique Da Silva,ST,2003-03-30,Oldham Athletic,English League Two,SUB,2020-08-01,England,Right,Medium/Medium,Lean (170-185),No


Select and show only the attributes that are numerical attributes

In [ ]:
numerical_df = players_21.select_dtypes(exclude=['object'])
numerical_df

,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,league_level,club_jersey_number,...,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
0,93,93,103500000.0,560000.0,33,170,72,241.0,1.0,10.0,...,75,96,32,35,24,6,11,15,14,8
1,92,92,63000000.0,220000.0,35,187,83,45.0,1.0,7.0,...,84,95,28,32,24,7,11,15,14,11
2,91,91,111000000.0,240000.0,31,184,80,21.0,1.0,9.0,...,88,88,35,42,19,15,6,12,8,10
3,91,91,132000000.0,270000.0,28,175,68,73.0,1.0,10.0,...,92,93,35,30,29,9,9,15,15,11
4,91,91,129000000.0,370000.0,29,181,70,10.0,1.0,17.0,...,84,91,68,65,53,15,13,5,10,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18939,47,52,70000.0,1000.0,21,177,70,112165.0,1.0,25.0,...,35,40,45,56,47,12,13,8,14,6
18940,47,53,70000.0,1000.0,21,174,68,112540.0,1.0,37.0,...,35,35,43,42,53,8,8,13,14,10
18941,47,47,45000.0,2000.0,28,185,79,111774.0,1.0,33.0,...,36,35,38,43,45,8,5,11,5,7
18942,47,67,130000.0,500.0,17,171,58,1920.0,4.0,27.0,...,50,45,18,11,13,11,13,9,9,6


In [ ]:
# Show to check which attributes have null values in the categorical dataset
categorical_df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   short_name        18944 non-null  object
 1   long_name         18944 non-null  object
 2   player_positions  18944 non-null  object
 3   dob               18944 non-null  object
 4   club_name         18719 non-null  object
 5   league_name       18719 non-null  object
 6   club_position     18719 non-null  object
 7   club_joined       17961 non-null  object
 8   nationality_name  18944 non-null  object
 9   preferred_foot    18944 non-null  object
 10  work_rate         18944 non-null  object
 11  body_type         18944 non-null  object
 12  real_face         18944 non-null  object
dtypes: object(13)
memory usage: 1.9+ MB


Select the attributes with null values based on the count of non-null rows and impute them using a forward fill

In [ ]:
# Select categorical attributes with null values
columns_to_fill = categorical_df.columns[categorical_df.count() < 18944]
columns_to_fill

Index(['club_name', 'league_name', 'club_position', 'club_joined'], dtype='object')

In [ ]:
# Forward fill attributes with na values
categorical_df[columns_to_fill] = categorical_df[columns_to_fill].fillna(method='ffill')

In [ ]:
# Check to see that there are no null values
categorical_df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   short_name        18944 non-null  object
 1   long_name         18944 non-null  object
 2   player_positions  18944 non-null  object
 3   dob               18944 non-null  object
 4   club_name         18944 non-null  object
 5   league_name       18944 non-null  object
 6   club_position     18944 non-null  object
 7   club_joined       18944 non-null  object
 8   nationality_name  18944 non-null  object
 9   preferred_foot    18944 non-null  object
 10  work_rate         18944 non-null  object
 11  body_type         18944 non-null  object
 12  real_face         18944 non-null  object
dtypes: object(13)
memory usage: 1.9+ MB


Create an object to encode the categorical variables and fit_transform all the cells inside of the data frame that contains just categorical variables

In [ ]:
# Object to encode categorical values
encoder = LabelEncoder()

In [ ]:
# label encode categorical values
categorical_df= categorical_df.apply(encoder.fit_transform)

In [ ]:
categorical_df

,short_name,long_name,player_positions,dob,club_name,league_name,club_position,club_joined,nationality_name,preferred_foot,work_rate,body_type,real_face
0,10059,10302,539,942,238,41,0,8,6,0,7,9,1
1,3261,3332,587,391,361,24,13,1135,123,1,1,9,1
2,14329,14687,560,1271,240,19,27,289,122,1,2,9,1
3,13007,12964,331,2438,461,17,14,888,20,1,2,9,1
4,8987,9626,10,2229,407,15,19,487,14,1,0,9,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18939,17684,18137,55,4982,138,6,28,1769,30,1,4,4,0
18940,6680,18648,149,4875,560,6,21,1769,30,1,4,1,0
18941,15978,18041,149,2366,608,6,21,1769,30,1,4,2,0
18942,16799,7152,560,6175,444,14,28,1769,48,1,8,1,0


Create object and impute the data frame containing numerical attributes to get rid of null values

In [ ]:
# Impute numerical attributes
imputer = si(strategy='median')

In [ ]:
# checking if null values exist in numerical attributes
numerical_df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 56 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   overall                      18944 non-null  int64  
 1   potential                    18944 non-null  int64  
 2   value_eur                    18707 non-null  float64
 3   wage_eur                     18719 non-null  float64
 4   age                          18944 non-null  int64  
 5   height_cm                    18944 non-null  int64  
 6   weight_kg                    18944 non-null  int64  
 7   club_team_id                 18719 non-null  float64
 8   league_level                 18719 non-null  float64
 9   club_jersey_number           18719 non-null  float64
 10  club_contract_valid_until    18719 non-null  float64
 11  nationality_id               18944 non-null  int64  
 12  weak_foot                    18944 non-null  int64  
 13  skill_moves     

In [ ]:
imputed_numerical = imputer.fit_transform(numerical_df)
imputed_numerical_df = pd.DataFrame(imputed_numerical, columns=numerical_df.columns)
imputed_numerical_df

,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,league_level,club_jersey_number,...,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
0,93.0,93.0,103500000.0,560000.0,33.0,170.0,72.0,241.0,1.0,10.0,...,75.0,96.0,32.0,35.0,24.0,6.0,11.0,15.0,14.0,8.0
1,92.0,92.0,63000000.0,220000.0,35.0,187.0,83.0,45.0,1.0,7.0,...,84.0,95.0,28.0,32.0,24.0,7.0,11.0,15.0,14.0,11.0
2,91.0,91.0,111000000.0,240000.0,31.0,184.0,80.0,21.0,1.0,9.0,...,88.0,88.0,35.0,42.0,19.0,15.0,6.0,12.0,8.0,10.0
3,91.0,91.0,132000000.0,270000.0,28.0,175.0,68.0,73.0,1.0,10.0,...,92.0,93.0,35.0,30.0,29.0,9.0,9.0,15.0,15.0,11.0
4,91.0,91.0,129000000.0,370000.0,29.0,181.0,70.0,10.0,1.0,17.0,...,84.0,91.0,68.0,65.0,53.0,15.0,13.0,5.0,10.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18939,47.0,52.0,70000.0,1000.0,21.0,177.0,70.0,112165.0,1.0,25.0,...,35.0,40.0,45.0,56.0,47.0,12.0,13.0,8.0,14.0,6.0
18940,47.0,53.0,70000.0,1000.0,21.0,174.0,68.0,112540.0,1.0,37.0,...,35.0,35.0,43.0,42.0,53.0,8.0,8.0,13.0,14.0,10.0
18941,47.0,47.0,45000.0,2000.0,28.0,185.0,79.0,111774.0,1.0,33.0,...,36.0,35.0,38.0,43.0,45.0,8.0,5.0,11.0,5.0,7.0
18942,47.0,67.0,130000.0,500.0,17.0,171.0,58.0,1920.0,4.0,27.0,...,50.0,45.0,18.0,11.0,13.0,11.0,13.0,9.0,9.0,6.0


Combine both the dataframes of the the categorical and numerical attributes into a single data frame

In [ ]:
new_players_21 = pd.concat([imputed_numerical_df, categorical_df], axis=1)
new_players_21

,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,league_level,club_jersey_number,...,dob,club_name,league_name,club_position,club_joined,nationality_name,preferred_foot,work_rate,body_type,real_face
0,93.0,93.0,103500000.0,560000.0,33.0,170.0,72.0,241.0,1.0,10.0,...,942,238,41,0,8,6,0,7,9,1
1,92.0,92.0,63000000.0,220000.0,35.0,187.0,83.0,45.0,1.0,7.0,...,391,361,24,13,1135,123,1,1,9,1
2,91.0,91.0,111000000.0,240000.0,31.0,184.0,80.0,21.0,1.0,9.0,...,1271,240,19,27,289,122,1,2,9,1
3,91.0,91.0,132000000.0,270000.0,28.0,175.0,68.0,73.0,1.0,10.0,...,2438,461,17,14,888,20,1,2,9,1
4,91.0,91.0,129000000.0,370000.0,29.0,181.0,70.0,10.0,1.0,17.0,...,2229,407,15,19,487,14,1,0,9,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18939,47.0,52.0,70000.0,1000.0,21.0,177.0,70.0,112165.0,1.0,25.0,...,4982,138,6,28,1769,30,1,4,4,0
18940,47.0,53.0,70000.0,1000.0,21.0,174.0,68.0,112540.0,1.0,37.0,...,4875,560,6,21,1769,30,1,4,1,0
18941,47.0,47.0,45000.0,2000.0,28.0,185.0,79.0,111774.0,1.0,33.0,...,2366,608,6,21,1769,30,1,4,2,0
18942,47.0,67.0,130000.0,500.0,17.0,171.0,58.0,1920.0,4.0,27.0,...,6175,444,14,28,1769,48,1,8,1,0


In [ ]:
# Confirm that there are no null values
new_players_21.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 69 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   overall                      18944 non-null  float64
 1   potential                    18944 non-null  float64
 2   value_eur                    18944 non-null  float64
 3   wage_eur                     18944 non-null  float64
 4   age                          18944 non-null  float64
 5   height_cm                    18944 non-null  float64
 6   weight_kg                    18944 non-null  float64
 7   club_team_id                 18944 non-null  float64
 8   league_level                 18944 non-null  float64
 9   club_jersey_number           18944 non-null  float64
 10  club_contract_valid_until    18944 non-null  float64
 11  nationality_id               18944 non-null  float64
 12  weak_foot                    18944 non-null  float64
 13  skill_moves     

In [ ]:
# change naming of variable due to preference
players_21 = new_players_21

## Feature Engineering

**Select the needed attributes to train the regression model with**

Here, I picked attributes with a correlation of more than |0.5| to ensure that we get the attributes with a strong correlation.

In [ ]:
correlation_vals = players_21.corr()['overall'] # check correlation of other attributes with overall
corr_attributes = correlation_vals[abs(correlation_vals.values) >= 0.5]  # which attributes have a strong correlation with the overall rating
corr_attributes.index

Index(['overall', 'potential', 'value_eur', 'wage_eur', 'release_clause_eur',
       'passing', 'dribbling', 'attacking_short_passing', 'movement_reactions',
       'power_shot_power', 'mentality_vision', 'mentality_composure'],
      dtype='object')

**Create new dataframe of needed attributes**

In [ ]:
needed_att_players_21 = players_21[corr_attributes.index]
needed_att_players_21

,overall,potential,value_eur,wage_eur,release_clause_eur,passing,dribbling,attacking_short_passing,movement_reactions,power_shot_power,mentality_vision,mentality_composure
0,93.0,93.0,103500000.0,560000.0,138400000.0,91.0,95.0,91.0,94.0,86.0,95.0,96.0
1,92.0,92.0,63000000.0,220000.0,75900000.0,81.0,89.0,82.0,95.0,94.0,82.0,95.0
2,91.0,91.0,111000000.0,240000.0,132000000.0,78.0,85.0,84.0,93.0,89.0,79.0,88.0
3,91.0,91.0,132000000.0,270000.0,166500000.0,86.0,94.0,87.0,91.0,80.0,90.0,93.0
4,91.0,91.0,129000000.0,370000.0,161000000.0,93.0,88.0,94.0,91.0,91.0,94.0,91.0
...,...,...,...,...,...,...,...,...,...,...,...,...
18939,47.0,52.0,70000.0,1000.0,57000.0,26.0,27.0,31.0,48.0,35.0,25.0,40.0
18940,47.0,53.0,70000.0,1000.0,72000.0,49.0,47.0,55.0,50.0,38.0,53.0,35.0
18941,47.0,47.0,45000.0,2000.0,47000.0,49.0,46.0,56.0,44.0,49.0,44.0,35.0
18942,47.0,67.0,130000.0,500.0,165000.0,40.0,53.0,45.0,53.0,50.0,49.0,45.0


**Scale dataset inputs only, by separating output attribute from the rest of the dataset**

This is to prevent overall score from being scaled. It was separated from the rest of the dataset and stored in a separate variable (overall_att) before going ahead to scale the input attributes

In [ ]:
overall_att = needed_att_players_21.overall

In [ ]:
model_data_players_21 = needed_att_players_21.drop(columns='overall')
model_data_players_21

,potential,value_eur,wage_eur,release_clause_eur,passing,dribbling,attacking_short_passing,movement_reactions,power_shot_power,mentality_vision,mentality_composure
0,93.0,103500000.0,560000.0,138400000.0,91.0,95.0,91.0,94.0,86.0,95.0,96.0
1,92.0,63000000.0,220000.0,75900000.0,81.0,89.0,82.0,95.0,94.0,82.0,95.0
2,91.0,111000000.0,240000.0,132000000.0,78.0,85.0,84.0,93.0,89.0,79.0,88.0
3,91.0,132000000.0,270000.0,166500000.0,86.0,94.0,87.0,91.0,80.0,90.0,93.0
4,91.0,129000000.0,370000.0,161000000.0,93.0,88.0,94.0,91.0,91.0,94.0,91.0
...,...,...,...,...,...,...,...,...,...,...,...
18939,52.0,70000.0,1000.0,57000.0,26.0,27.0,31.0,48.0,35.0,25.0,40.0
18940,53.0,70000.0,1000.0,72000.0,49.0,47.0,55.0,50.0,38.0,53.0,35.0
18941,47.0,45000.0,2000.0,47000.0,49.0,46.0,56.0,44.0,49.0,44.0,35.0
18942,67.0,130000.0,500.0,165000.0,40.0,53.0,45.0,53.0,50.0,49.0,45.0


Create Standard Scaler object (sc)

In [ ]:
sc = StandardScaler()

In [ ]:
# scale input values of data
scaled_data = sc.fit_transform(model_data_players_21)
ready_data_players_21 = pd.DataFrame(scaled_data, columns=model_data_players_21.columns)
ready_data_players_21

,potential,value_eur,wage_eur,release_clause_eur,passing,dribbling,attacking_short_passing,movement_reactions,power_shot_power,mentality_vision,mentality_composure
0,3.586563,13.071234,27.845078,13.676627,3.482519,3.410670,2.216206,3.554438,2.119026,3.000047,3.137573
1,3.422893,7.810099,10.660644,7.310503,2.451148,2.778572,1.598583,3.664174,2.719163,2.052781,3.055051
2,3.259222,14.045519,11.671493,13.024735,2.141737,2.357173,1.735833,3.444701,2.344077,1.834181,2.477402
3,3.259222,16.773515,13.187767,16.538836,2.966834,3.305320,1.941707,3.225227,1.668922,2.635714,2.890009
4,3.259222,16.383801,18.242012,15.978617,3.688793,2.673222,2.422080,3.225227,2.494112,2.927180,2.724966
...,...,...,...,...,...,...,...,...,...,...,...
18939,-3.123941,-0.364795,-0.408153,-0.414712,-3.221388,-3.753112,-1.901280,-1.493455,-1.706851,-2.100615,-1.483625
18940,-2.960270,-0.364795,-0.408153,-0.413184,-0.849236,-1.646118,-0.254285,-1.273981,-1.481799,-0.060350,-1.896232
18941,-3.942295,-0.368043,-0.357611,-0.415730,-0.849236,-1.751467,-0.185661,-1.932402,-0.656610,-0.716150,-1.896232
18942,-0.668878,-0.357001,-0.433425,-0.403711,-1.777469,-1.014019,-0.940533,-0.944771,-0.581593,-0.351817,-1.071018


## Model development and testing

**Using the test/train split and splitting the X(input attributes) and Y(Output attribute) to train and test the models**

In [ ]:
X = ready_data_players_21
Y = overall_att

In [ ]:
# test/train split for cross-validation
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
Xtrain.shape

(15155, 11)

**Train, test, and measure accuracy of RandomForestRegressor model with training split of dataset and testing split of dataset**

Create a RandomForestRegressor model (rf_model).
Use the test data-set ( Xtest ) to make predictions and then check the accuracy using mean absolute error

In [ ]:
# Instantiate Regression model
rf_model = RandomForestRegressor(n_estimators=2000, max_depth=1000, n_jobs=-1, random_state=42)

In [ ]:
# train the regression model
rf_model.fit(Xtrain, Ytrain)

RandomForestRegressor(max_depth=1000, n_estimators=2000, n_jobs=-1,
                      random_state=42)

In [ ]:
# use model for prediction
model_prediction = rf_model.predict(Xtest)

In [ ]:
# Measure accuracy
mean_abs_err_results = mean_absolute_error(Ytest, model_prediction)
mean_abs_err_results

0.6114444444444443

**Train, test and measure accuracy of AdaBoostRegressor model with training split of dataset and testing split of dataset**

Create an AdaBoostRegressor with a base model as a decisiontreeregressor

In [ ]:
base_model = DecisionTreeRegressor(max_depth=1000)

In [ ]:
adaboost_regressor = AdaBoostRegressor(base_model, n_estimators=1000, random_state=42)

adaboost_regressor.fit(Xtrain, Ytrain)

AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=1000),
                  n_estimators=1000, random_state=42)

In [ ]:
ada_predict = adaboost_regressor.predict(Xtest)
mae = mean_absolute_error(Ytest, ada_predict)
mae

0.4333597255212457

**Train, test, and measure accuracy of Gradient Boosting model with training split of dataset and testing split of dataset**

Create  and train a GradientBoosting model (gb_model)
Use the test data-set ( Xtest ) to make predictions and then check the accuracy using mean absolute error

In [ ]:
# Instantiate GradientBoosting model
gb_model = GradientBoostingRegressor(n_estimators=50, random_state=42)

In [ ]:
# Train the GradientBoosting model
gb_model.fit(Xtrain, Ytrain)

GradientBoostingRegressor(n_estimators=50, random_state=42)

In [ ]:
# use GradientBoosting model for prediction
gb_prediction = gb_model.predict(Xtest)

In [ ]:
gb_mean_abs_err = mean_absolute_error(Ytest, gb_prediction)
gb_mean_abs_err

1.236073643187885

**Train, test, and measure accuracy of XGboost model with training split of dataset and testing split of daaset**

Create  and train a XGboost model (xg_model).
Use the test data-set ( Xtest ) to make predictions and then check the accuracy using mean absolute error.

In [ ]:
# Create model that looks to minimize mean squared error as much as possible
xg_model = xgb.XGBRegressor(objective="reg:squarederror", n_estimators=50)

In [ ]:
# train model
xg_model.fit(Xtrain, Ytrain)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=50, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [ ]:
# predict with model
xg_prediction = xg_model.predict(Xtest)

In [ ]:
xg_mean_abs_err = mean_absolute_error(Ytest, xg_prediction)
xg_mean_abs_err

0.7776270242440855

**Train, test and measure accuracy of VotingRegressor ensemble using all previously used models**

VotingRegressor is used as an ensemble model and the voting method is soft because it gives better results.
RandomForest is given a weight of 0.8 because it consistently performs better than the others. The Gradient Boosting ('gradientboosing') and XGBoost ('xgb') models are given lower weights of 0.1 each, suggesting that their contributions are considered less significant relative to the Random Forest. These weights were assigned based on empirical experimentation and observing how different weight combinations impact the overall ensemble performance.

In [ ]:
# Create model instance
soft_ensemble = VotingRegressor(estimators=[
    ('randomforest', rf_model),
    ('gradientboosing', gb_model),
    ('xgb', xg_model)
], weights=[0.8, 0.1, 0.1])

In [ ]:
soft_ensemble.fit(Xtrain, Ytrain)

VotingRegressor(estimators=[('randomforest',
                             RandomForestRegressor(max_depth=1000,
                                                   n_estimators=2000, n_jobs=-1,
                                                   random_state=42)),
                            ('gradientboosing',
                             GradientBoostingRegressor(n_estimators=50,
                                                       random_state=42)),
                            ('xgb',
                             XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          e...
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=None, max_bin=None,
                                          max_cat_threshold=None,
                                          max_cat_to_onehot=None,
                                          max_delta_step=None, max_depth=None,
                                          max_leaves=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          multi_strategy=None, n_estimators=50,
                                          n_jobs=None, num_parallel_tree=None,
                                          random_state=None, ...))],
                weights=[0.8, 0.1, 0.1])

In [ ]:
ensemble_prediction = soft_ensemble.predict(Xtest)

Check accuracy of the VotingRegressor model using mean absolute error

In [ ]:
ensemble_mean_abs_err = mean_absolute_error(Ytest, ensemble_prediction)
ensemble_mean_abs_err

0.6416101353103437

**Compare the different absolute error values of the different models used and select the model to go with**

In [ ]:
print("RandomForest Model: ", mean_abs_err_results)
print("RandomForest Model: ", mae)
print("GradientBoosting Model: ", gb_mean_abs_err)
print("XGBBoost Model: ", xg_mean_abs_err)
print("SoftEnsemble Model: ", ensemble_mean_abs_err)

RandomForest Model:  0.6114444444444443
RandomForest Model:  0.4333597255212457
GradientBoosting Model:  1.236073643187885
XGBBoost Model:  0.7776270242440855
SoftEnsemble Model:  0.6416101353103437


**From observation, the AdaBoostRegressor Model is the more accurate one so it's the preference to fine tune**

Fine tune the AdaBoostRegressor model using the GridSearchCV to find the weights needed to give the highest level of accuracy (best score).
The parameter grid for the GridSearchCv, specifies that the 'n_estimators' parameter is being tuned along with the max_depth, and learning_rate. The negative mean absolute error is used because GridSearchCv seeks to maximize the scoring metric, and we want to minimize mean absolute error.

## Fine Tuning and hyperparameterisation

In [ ]:
# Fine tune hyperparameters of model
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2000, num = 10)]
max_depth = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
min_samples_split = [2, 5, 10, 15, 20]
param_grid = {'n_estimators':n_estimators, 'max_depth':max_depth, 'min_samples_split':min_samples_split}

In [ ]:
cv=KFold(n_splits=3)
# random_search = RandomizedSearchCV(estimator=rf_model,
#                                    param_distributions=param_grid,
#                                    n_iter=10, random_state=7, cv=cv,
#                                    scoring= 'neg_mean_absolute_error')

In [ ]:
random_search = GridSearchCV(rf_model,param_grid=param_grid,cv=cv,scoring="neg_mean_absolute_error")

In [ ]:
cv=KFold(n_splits=3)
n_estimators = [1000, 2000, 4000]
learning_rate = [0.1, 0.2, 0.3]
param_grid = {'n_estimators':n_estimators,'learning_rate':learning_rate}

In [ ]:
random_search = GridSearchCV(adaboost_regressor,param_grid=param_grid,cv=cv,scoring="neg_mean_absolute_error", n_jobs=3)

Find the best value for the mean absolute error and the parameter value (n_estimators) needed to provide this best score

In [ ]:
random_search.fit(X, Y)

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


GridSearchCV(cv=KFold(n_splits=3, random_state=None, shuffle=False),
             estimator=AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=1000),
                                         n_estimators=1000, random_state=42),
             n_jobs=3,
             param_grid={'learning_rate': [0.1, 0.2, 0.3],
                         'n_estimators': [1000, 2000, 4000]},
             scoring='neg_mean_absolute_error')

In [ ]:
print('best estimator: ', random_search.best_estimator_)
print('best param: ', random_search.best_params_)

best estimator:  AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=1000),
                  learning_rate=0.2, n_estimators=2000, random_state=42)
best param:  {'learning_rate': 0.2, 'n_estimators': 2000}


**Implement hyper parameter after fine tuning.**

Create a AdaBoostRegressor model with 1000 depth decision tree model, and 2000 as the number of estimators

In [ ]:
base_model_2 = DecisionTreeRegressor(max_depth=1000)
model_2 = AdaBoostRegressor(base_model_2, n_estimators=2000, learning_rate=0.2, random_state=42)
model_2.fit(Xtrain, Ytrain)

model_prediction_2 = model_2.predict(Xtest)

Check accuracy of model using mean absolute error

In [ ]:
mean_abs_err_results_2 = mean_absolute_error(Ytest, model_prediction_2)
mean_abs_err_results_2

0.4336236474003695

## Testing model on New data

**Prepare players_22 dataset for testing model on new unseen data to check for overfitting**

Pick out only the needed attributes from the dataset

In [ ]:
# Prepping players_22 to use for testing
df_22 = players_22[corr_attributes.index]
df_22

,overall,potential,value_eur,wage_eur,release_clause_eur,passing,dribbling,attacking_short_passing,movement_reactions,power_shot_power,mentality_vision,mentality_composure
0,93,93,78000000.0,320000.0,144300000.0,91.0,95.0,91,94,86,95,96
1,92,92,119500000.0,270000.0,197200000.0,79.0,86.0,85,93,90,81,88
2,91,91,45000000.0,270000.0,83300000.0,80.0,88.0,80,94,94,76,95
3,91,91,129000000.0,270000.0,238700000.0,86.0,94.0,86,89,80,90,93
4,91,91,125500000.0,350000.0,232200000.0,93.0,88.0,94,91,91,94,89
...,...,...,...,...,...,...,...,...,...,...,...,...
19234,47,52,70000.0,1000.0,114000.0,46.0,48.0,50,53,46,43,37
19235,47,59,110000.0,500.0,193000.0,50.0,46.0,51,49,49,49,47
19236,47,55,100000.0,500.0,175000.0,45.0,49.0,49,46,50,46,36
19237,47,60,110000.0,500.0,239000.0,36.0,48.0,38,48,48,40,47


Check if null values exist

In [ ]:
df_22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   overall                  19239 non-null  int64  
 1   potential                19239 non-null  int64  
 2   value_eur                19165 non-null  float64
 3   wage_eur                 19178 non-null  float64
 4   release_clause_eur       18063 non-null  float64
 5   passing                  17107 non-null  float64
 6   dribbling                17107 non-null  float64
 7   attacking_short_passing  19239 non-null  int64  
 8   movement_reactions       19239 non-null  int64  
 9   power_shot_power         19239 non-null  int64  
 10  mentality_vision         19239 non-null  int64  
 11  mentality_composure      19239 non-null  int64  
dtypes: float64(5), int64(7)
memory usage: 1.8 MB


Remove the null values by imputing the values in the dataset

In [ ]:
imputed_df_22 = imputer.fit_transform(df_22)
imputed_df_22 = pd.DataFrame(imputed_df_22, columns=df_22.columns)
imputed_df_22

,overall,potential,value_eur,wage_eur,release_clause_eur,passing,dribbling,attacking_short_passing,movement_reactions,power_shot_power,mentality_vision,mentality_composure
0,93.0,93.0,78000000.0,320000.0,144300000.0,91.0,95.0,91.0,94.0,86.0,95.0,96.0
1,92.0,92.0,119500000.0,270000.0,197200000.0,79.0,86.0,85.0,93.0,90.0,81.0,88.0
2,91.0,91.0,45000000.0,270000.0,83300000.0,80.0,88.0,80.0,94.0,94.0,76.0,95.0
3,91.0,91.0,129000000.0,270000.0,238700000.0,86.0,94.0,86.0,89.0,80.0,90.0,93.0
4,91.0,91.0,125500000.0,350000.0,232200000.0,93.0,88.0,94.0,91.0,91.0,94.0,89.0
...,...,...,...,...,...,...,...,...,...,...,...,...
19234,47.0,52.0,70000.0,1000.0,114000.0,46.0,48.0,50.0,53.0,46.0,43.0,37.0
19235,47.0,59.0,110000.0,500.0,193000.0,50.0,46.0,51.0,49.0,49.0,49.0,47.0
19236,47.0,55.0,100000.0,500.0,175000.0,45.0,49.0,49.0,46.0,50.0,46.0,36.0
19237,47.0,60.0,110000.0,500.0,239000.0,36.0,48.0,38.0,48.0,48.0,40.0,47.0


In [ ]:
imputed_df_22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   overall                  19239 non-null  float64
 1   potential                19239 non-null  float64
 2   value_eur                19239 non-null  float64
 3   wage_eur                 19239 non-null  float64
 4   release_clause_eur       19239 non-null  float64
 5   passing                  19239 non-null  float64
 6   dribbling                19239 non-null  float64
 7   attacking_short_passing  19239 non-null  float64
 8   movement_reactions       19239 non-null  float64
 9   power_shot_power         19239 non-null  float64
 10  mentality_vision         19239 non-null  float64
 11  mentality_composure      19239 non-null  float64
dtypes: float64(12)
memory usage: 1.8 MB


In [ ]:
Y = imputed_df_22.overall
X = imputed_df_22.drop(columns='overall')

In [ ]:
X_scaled = sc.fit_transform(X)
X = pd.DataFrame(X_scaled, columns=X.columns)
X

,potential,value_eur,wage_eur,release_clause_eur,passing,dribbling,attacking_short_passing,movement_reactions,power_shot_power,mentality_vision,mentality_composure
0,3.601780,9.889397,15.996581,9.589016,3.539198,3.542596,2.217457,3.599846,2.139433,3.006228,3.131025
1,3.437470,15.350116,13.424792,13.234253,2.275622,2.554865,1.803392,3.489252,2.442649,1.980597,2.473077
2,3.273160,5.547138,13.424792,5.385624,2.380920,2.774360,1.458338,3.599846,2.745866,1.614300,3.048782
3,3.273160,16.600160,13.424792,16.093937,3.012708,3.432848,1.872403,3.046874,1.684607,2.639931,2.884295
4,3.273160,16.139617,17.539654,15.646035,3.749794,2.774360,2.424490,3.268063,2.518454,2.932969,2.555321
...,...,...,...,...,...,...,...,...,...,...,...
19234,-3.134932,-0.364913,-0.411430,-0.346563,-1.199211,-1.615558,-0.611986,-0.934526,-0.892735,-0.803260,-1.721343
19235,-1.984762,-0.359649,-0.437148,-0.341119,-0.778020,-1.835054,-0.542976,-1.376903,-0.665323,-0.363704,-0.898908
19236,-2.642002,-0.360965,-0.437148,-0.342360,-1.304509,-1.505810,-0.680997,-1.708687,-0.589519,-0.583482,-1.803587
19237,-1.820452,-0.359649,-0.437148,-0.337949,-2.252191,-1.615558,-1.440116,-1.487498,-0.741127,-1.023038,-0.898908


**Use model to predict with new data and check accuracy**

In [ ]:
# Model on new players_22 data
players_22_prediction = model_2.predict(X)
mean_absolute_error(Y, players_22_prediction)

1.3092156556993606

## Save Model

**Saving Model and Scaler object for deployment purposes**

In [ ]:
import pickle

Both the scaler and model objects are saved in files. Scaler objects are used to ensure that the input values of users in the deployed site are scaled using the same standard deviation values as the one with which the model was trained with.

In [ ]:
# Scaler object as file to scale user input with the necessary parameters
pickle.dump(sc, open('/content/drive/MyDrive/Colab Notebooks/new_scaler_parameters.pkl', 'wb'))

# Model object as file to predict the scaled values after user input
pickle.dump(model_2, open('/content/drive/MyDrive/Colab Notebooks/new_player_ratings_model.pkl', 'wb'))